In [12]:
import os
path = "/home/marta/Documenti/eeg-ml-thesis/"
os.chdir(path)

In [13]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import r_pca 
import scipy.io


In [14]:
def create_dataset(subject_list, window, overlap, num_columns=16, num_classes=2):

    x_data = np.empty((0, window, num_columns))
    y_data = np.empty((0, 1))  # Labels
    subj_inputs = []  # Tracks number of windows per subject
    
    dataset_dir = '/home/marta/Documenti/eeg_rnn_repo/rnn-eeg-ad/eeg2'
    # print('\n### Creating dataset')
    tot_rows = 0
    
    # for subject_id, category_label in subject_list:
    subject_id = subject_list[0]
    category_label = subject_list[1]
    
    # print(f"aaaaaaaaaaaaaa{subject_id}")
    # print(f"bbbbbbbbbbbbb{category_label}")
    subj_inputs.append(0)  # Initialize window count for this subject
    
    # Load EEG data
    file_path = f"{dataset_dir}/S{subject_id}_{category_label}.npz"
    eeg = np.load(file_path)['eeg'].T  # Transpose if necessary to get [samples, channels]
    
    # Scale EEG data
    scaler = StandardScaler()
    eeg = scaler.fit_transform(eeg)
    
    assert eeg.shape[1] == num_columns, f"Expected {num_columns} channels, got {eeg.shape[1]}"
    
    # Calculate number of windows
    num_windows = 0
    i = 0
    while i + window <= len(eeg):
        i += (window - overlap)
        num_windows += 1
    
    # Preallocate x_data for this subject
    x_data_part = np.empty((num_windows, window, num_columns))
    
    # Extract windows
    i = 0
    for w in range(num_windows):
        x_data_part[w] = eeg[i:i + window]
        i += (window - overlap)
    
    # Update x_data and y_data
    x_data = np.vstack((x_data, x_data_part))
    y_data = np.vstack((y_data, np.full((num_windows, 1), (category_label == 'AD'))))  # Binary label
    subj_inputs[-1] = num_windows
    tot_rows += len(eeg)
    
    # print(f"Total samples: {tot_rows}")
    # print(f"x_data shape: {x_data.shape}")
    # print(f"y_data shape: {y_data.shape}")
    # print(f"Windows per subject: {subj_inputs}")
    # print(f"Class distribution: {[np.sum(y_data == cl) for cl in range(num_classes)]}")
    
    return x_data, y_data, subj_inputs


In [27]:
class EegDataset(Dataset):
    
    def __init__(self, 
                 file_paths, 
                #  labels, 
                 create_dataset_crop, 
                 window, 
                 overlap):
        
        super().__init__()
        self.file_paths = file_paths
        # self.labels = labels
        self.create_dataset_crop = create_dataset_crop
        self.window = window
        self.overlap = overlap
        
        self.crops_index = self._compute_crops_index()
    
    def _compute_crops_index(self):
        crops_index = []
        for file_idx, (file_path) in enumerate(self.file_paths):
            # print(f"file_path: {file_path}")
            crops, _, _ = self.create_dataset_crop(file_path, self.window, self.overlap)
            
            num_crops = len(crops)
            
            crops_index.extend([(file_idx, crop_idx) for crop_idx in range(num_crops)])
            
        return crops_index
    
    def __len__(self):
        return len(self.crops_index)
    
    def __getitem__(self, idx):
        
        file_idx, crop_idx = self.crops_index[idx]
        file_path = self.file_paths[file_idx]
        
        crops, labels, _ = self.create_dataset_crop(file_path, self.window, self.overlap)
        x_data_reduced, Vpca = reduce_matrix(crops, None)
        labels = adjust_size(x_data_reduced, labels)
        # print(np.unique(label[0]))
        # print(label.shape)
        crop = x_data_reduced[crop_idx]
        label = labels[0] 
        
        label = torch.tensor(label).float().squeeze().unsqueeze(0)        
        # label = self.labels[file_idx]
        
        return torch.tensor(crop).float(), label
    


# create_dataset

# apply pca on the crops




dataset = EegDataset(file_paths=train_val_subjects,
                    create_dataset_crop=create_dataset,
                    window=128,
                    overlap=25)

test_dataset = EegDataset(file_paths=test_subject_list,
                    create_dataset_crop=create_dataset,
                    window=128,
                    overlap=25)

train_indices, val_indices = split_train_val(dataset, test_size=0.2)
train_loader = DataLoader(Subset(dataset, train_indices), batch_size=32, shuffle=True)
val_loader = DataLoader(Subset(dataset, val_indices), batch_size=32, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size = 32)

In [36]:
def create_dataset(window, overlap, decimation_factor = 0):
  # Create the input and target data from dataset,
  # according to window and overlap
  # new dataset 4 dec 2021
  # 15 N, 20 AD (resulting indexes: N = 0..14, AD = 15..34)
  #Common signals: ['EEG Fp1', 'EEG Fp2', 'EEG F7', 'EEG F3', 'EEG F4', 'EEG F8', 'EEG T3', 'EEG C3', 'EEG C4', 'EEG T4', 'EEG T5', 'EEG P3', 'EEG P4', 'EEG T6', 'EEG O1', 'EEG O2']

  np.random.seed(42)
  dataset_dir = '/home/marta/Documenti/eeg_rnn_repo/rnn-eeg-ad/eeg2'
  subj_list = tuple((f'{i:02d}', 'N') for i in range(1, 16)) + tuple((f'{i:02d}', 'AD') for i in range(1, 21))
  print(subj_list)
  num_columns = 16

  x_data = np.empty((0, window, num_columns))
  y_data = np.empty((0, 1))  # labels
  subj_inputs = []  # number of inputs for every subject
  print('\n### creating dataset')
  tot_rows = 0
  for subject in subj_list:
    subj_inputs.append(0)
    category = ('N', 'AD').index(subject[1])
    eeg = np.load(f'{dataset_dir}/S{subject[0]}_{subject[1]}.npz')['eeg'].T
    # if spikes: eeg = set_holes(eeg, spikes)
    #scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = StandardScaler()
    eeg = scaler.fit_transform(eeg)
    assert(eeg.shape[1] == num_columns)
    tot_rows += len(eeg)
    # decimation (optional)
    if decimation_factor:
      eeg2 = np.empty((eeg.shape[0] // decimation_factor, eeg.shape[1]))
      for col in range(0, num_columns):
        #tmp = scipy.signal.decimate(fusion[:, col], decimation_factor)
        tmp = eeg[:, col][::decimation_factor]  # simpler method
        eeg2[:, col] = tmp[:len(eeg2)]
      eeg = eeg2
    # windowing
    # compute number of windows (lazy way)
    i = 0
    num_w = 0
    while i + window  <= len(eeg):
      i += (window - overlap)
      num_w += 1
    # compute actual windows
    x_data_part = np.empty((num_w, window, num_columns))  # preallocate
    i = 0
    for w in range(0, num_w):
      x_data_part[w] = eeg[i:i + window]
      i += (window - overlap)
      if False: # watermark provenience of every window
        for cc in range(0, num_columns):
          x_data_part[w, 0, cc] = 1000 * (len(subj_inputs) - 1) + cc
    x_data = np.vstack((x_data, x_data_part))
    y_data = np.vstack((y_data, np.full((num_w, 1), category)))
    subj_inputs[-1] += num_w

  print('\ntot samples:', tot_rows)
  print('x_data:', x_data.shape)
  print('y_data:', y_data.shape)
  print('windows per subject:', subj_inputs)
  #print('class distribution:', [np.sum(y_data == cl) for cl in range(0, num_classes)])

  return x_data, y_data, subj_inputs

In [15]:
def oversampling(x_data, y_data, num_classes=2):
  # Duplicate inputs with classes occurring less, so to have a more balanced distribution.
  # It operates on single data windows, so use it on data that have already been split
  #  by subject (typically only on training data).
  x_data_over = x_data.copy()
  y_data_over = y_data.copy()
  occurr = [np.sum(y_data == cl) for cl in range(0, num_classes)]
  for cl in range(0, num_classes):
    if occurr[cl] == max(occurr):
      continue
    mask = y_data[:, 0] == cl
    x_dup = x_data[mask].copy()
    y_dup = y_data[mask].copy()
    while occurr[cl] < max(occurr):
      x_dup_jitter = x_dup + np.random.normal(scale=0.03, size=x_dup.shape)
      how_many = min(len(y_dup), max(occurr) - occurr[cl])
      x_data_over = np.vstack((x_data_over, x_dup_jitter[:how_many]))
      y_data_over = np.vstack((y_data_over, y_dup[:how_many]))
      occurr[cl] += how_many
  return x_data_over, y_data_over

In [7]:
## CREATE DATASET FOR ALESSANDRINI 

subjs_train_perm = ( (tuple(i for i in range(2, 15)) + tuple(i for i in range(18, 35)), ()), )
subjs_test = (0, 1, 15, 16, 17)  # 2 for N, 3 for AD
#x, y, subjs = create_dataset(train_val_subjects, 128, 25)

## Codice da usare

In [18]:
DATASET_DIR = "/home/marta/Documenti/eeg_rnn_repo/rnn-eeg-ad/eeg2"
# save_dir = "/home/marta/Documenti/eeg-ml-thesis/output"

def precompute_crops(subject_list, window, overlap, num_columns=16, train_dataset=None):
    

    if train_dataset == True:
        save_dir = "/home/marta/Documenti/eeg-ml-thesis/alessandrini-train"
        os.makedirs(save_dir, exist_ok=True)
    elif train_dataset == False:
        save_dir = "/home/marta/Documenti/eeg-ml-thesis/alessandrini-test"
        os.makedirs(save_dir, exist_ok=True)

    for subject_id, category_label in subject_list:
        file_path = f"{DATASET_DIR}/S{subject_id}_{category_label}.npz"
        save_path = f"{save_dir}/S{subject_id}_{category_label}_crops.npz"

        # if os.path.exists(save_path): 
        #    print(f"Skipping {subject_id}, crops already exist.")
        #    continue

        eeg = np.load(file_path)['eeg'].T 

        scaler = StandardScaler()
        eeg = scaler.fit_transform(eeg)

        num_windows = (len(eeg) - window) // (window - overlap) + 1
        x_data = np.empty((num_windows, window, num_columns))

        i = 0
        for w in range(num_windows):
            x_data[w] = eeg[i:i + window]
            i += (window - overlap)

        y_data = np.full((num_windows, 1), (category_label == 'AD')) 

        np.savez(save_path, x_data=x_data, y_data=y_data)
        # print(f"Saved crops for {subject_id} at {save_path}")

window = 256
overlap = window // 2
subj_list = (
    tuple((f'{i:02d}', 'N') for i in range(1, 16)) +  # normal subjects, S01 to S15
    tuple((f'{i:02d}', 'AD') for i in range(1, 21))   # alzheimer's subjects, S01 to S20
)
subjs_test = (0, 1, 15, 16, 17)  
test_subject_list = [subj_list[i] for i in subjs_test]
train_val_subjects = [subj for i, subj in enumerate(subj_list) if i not in subjs_test]   
precompute_crops(train_val_subjects, window=256, overlap=overlap, train_dataset=True)
precompute_crops(test_subject_list, window=256, overlap=overlap, train_dataset=False)

Saved crops for 03 at /home/marta/Documenti/eeg-ml-thesis/alessandrini-train/S03_N_crops.npz
Saved crops for 04 at /home/marta/Documenti/eeg-ml-thesis/alessandrini-train/S04_N_crops.npz
Saved crops for 05 at /home/marta/Documenti/eeg-ml-thesis/alessandrini-train/S05_N_crops.npz
Saved crops for 06 at /home/marta/Documenti/eeg-ml-thesis/alessandrini-train/S06_N_crops.npz
Saved crops for 07 at /home/marta/Documenti/eeg-ml-thesis/alessandrini-train/S07_N_crops.npz
Saved crops for 08 at /home/marta/Documenti/eeg-ml-thesis/alessandrini-train/S08_N_crops.npz
Saved crops for 09 at /home/marta/Documenti/eeg-ml-thesis/alessandrini-train/S09_N_crops.npz
Saved crops for 10 at /home/marta/Documenti/eeg-ml-thesis/alessandrini-train/S10_N_crops.npz
Saved crops for 11 at /home/marta/Documenti/eeg-ml-thesis/alessandrini-train/S11_N_crops.npz
Saved crops for 12 at /home/marta/Documenti/eeg-ml-thesis/alessandrini-train/S12_N_crops.npz
Saved crops for 13 at /home/marta/Documenti/eeg-ml-thesis/alessandrini

In [27]:
def split_train_val(dataset, test_size=0.2, random_state=42):
    train_indices, val_indices = train_test_split(
        range(len(dataset.crops_index)), 
        test_size=test_size,
        random_state=random_state
    )
    return train_indices, val_indices 

def pca_reduction(A, tol, comp = 0):
  rpca = False
  rpca_mu = 0
  multiscale_pca = False

  assert(len(A.shape) == 2)
  dmin = min(A.shape)
  if rpca:
    r = r_pca.R_pca(A, mu = rpca_mu)
    print('Auto tol:', 1e-7 * r.frobenius_norm(r.D), 'used tol:', tol)
    print('mu', r.mu, 'lambda', r.lmbda)
    L, S = r.fit(tol = tol, max_iter = 10, iter_print = 1)
    global norm_s
    norm_s = np.linalg.norm(S, ord='fro')  # for debug
    print('||A,L,S||:', np.linalg.norm(A, ord='fro'), np.linalg.norm(L, ord='fro'), np.linalg.norm(S, ord='fro'))
    #np.savez_compressed('rpca.npz', pre = A, post = L)
  elif multiscale_pca:
    print('MSPCA...')
    #ms = mspca.MultiscalePCA()
    #L = ms.fit_transform(A, wavelet_func='sym4', threshold=0.1, scale = True )
    print('saving MAT file and calling Matlab...')
    scipy.io.savemat('mspca.mat', {'A': A}, do_compression = True)
    os.system('matlab -batch "mspca(\'mspca.mat\')"')
    L = scipy.io.loadmat('mspca.mat')['L'] 
  else:
    
    L = A
  U, lam, V = np.linalg.svd(L, full_matrices = False)  # V is transposed
  assert(U.shape == (A.shape[0], dmin) and lam.shape == (dmin,) and V.shape == (dmin, A.shape[1]))
  #np.savetxt('singular_values.csv', lam)
  lam_trunc = lam[lam > 0.015 * lam[0]]  # magic number
  p = comp if comp else len(lam_trunc)
  assert(p <= dmin)
  #print('PCA truncation', dmin, '->', p)
  return L, V.T[:,:p]

def reduce_matrix(A, V):
  # (N, w, 16) → (N, 16, w) → ((N*16), w) → compute V
  # (N, 16, w) * V → transpose again last dimensions
  B = np.swapaxes(A, 1, 2)  # (N, 16, w)
  C = B.reshape((-1, B.shape[2]))  # ((N*16), w)
  if V is None:
    L, V = pca_reduction(C, 5e-6, comp = 50)
  B = C @ V  # ((N*16), p)
  B = B.reshape((A.shape[0], A.shape[2], B.shape[1]))  # (N, 16, p)
  return np.swapaxes(B, 1, 2), V  # B = (N, p, 16)

def adjust_size(x, y):
  # when flattening the data matrix on the first dimension, y must be made compatible
  if len(x) == len(y): return y
  factor = len(x) // len(y)
  ynew = np.empty((len(x), 1))
  for i in range(0, len(y)):
    ynew[i * factor : (i + 1) * factor] = y[i]
  return ynew


In [19]:
path = "/home/marta/Documenti/eeg-ml-thesis/alessandrini-train"

x_all = []
y_all = []

for file in os.listdir(path):
    if file.endswith(".npz"):  
        file_path = os.path.join(path, file)
        data = np.load(file_path)
        
        x_all.append(data['x_data'])  
        y_all.append(data['y_data'])  

x_all = np.vstack(x_all)  
y_all = np.vstack(y_all)

print(f"Original dataset size: {x_all.shape}, Labels distribution: {np.bincount(y_all.flatten())}")

x_all_over, y_all_over = oversampling(x_all, y_all)

print(f"Oversampled dataset size: {x_all_over.shape}, Labels distribution: {np.bincount(y_all_over.flatten())}")

x_all_over = torch.tensor(x_all_over).float()
y_all_over = torch.tensor(y_all_over).float()


S07_N_crops.npz
S05_N_crops.npz
S11_N_crops.npz
S04_AD_crops.npz
S06_N_crops.npz
S12_N_crops.npz
S11_AD_crops.npz
S10_AD_crops.npz
S05_AD_crops.npz
S09_N_crops.npz
S09_AD_crops.npz
S08_N_crops.npz
S06_AD_crops.npz
S18_AD_crops.npz
S15_AD_crops.npz
S15_N_crops.npz
S04_N_crops.npz
S10_N_crops.npz
S12_AD_crops.npz
S07_AD_crops.npz
S03_N_crops.npz
S13_AD_crops.npz
S13_N_crops.npz
S20_AD_crops.npz
S16_AD_crops.npz
S08_AD_crops.npz
S14_AD_crops.npz
S17_AD_crops.npz
S19_AD_crops.npz
S14_N_crops.npz
Original dataset size: (40602, 256, 16), Labels distribution: [16290 24312]
Oversampled dataset size: (48624, 256, 16), Labels distribution: [24312 24312]


In [28]:
x_data_train, x_data_val, y_data_train, y_data_val = train_test_split(x_all_over, y_all_over, train_size = 0.8, random_state=42, shuffle=True)

x_data_train, Vpca = reduce_matrix(x_data_train, None)
y_data_train = adjust_size(x_data_train, y_data_train)

x_data_val, _ = reduce_matrix(x_data_val, Vpca)
y_data_val = adjust_size(x_data_val, y_data_val)

In [30]:
class AlessandriniEegDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

train_dataset = AlessandriniEegDataset(x_data_train, y_data_train)
val_dataset = AlessandriniEegDataset(x_data_val, y_data_val)

#train_indices, val_indices = split_train_val(dataset_over, test_size=0.2)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)


In [31]:
for i, (x, y) in enumerate(train_loader):
    print(x.shape)
    print(y.shape)
    break

torch.Size([32, 50, 16])
torch.Size([32, 1])


## Forse serve per miltiadous

In [ ]:
class EegDataset(Dataset):
    def __init__(self, file_paths):
        super().__init__()
        self.file_paths = file_paths
        self.crops_index = self._compute_crops_index()
    
    def _compute_crops_index(self):
        crops_index = []
        for file_idx, file_path in enumerate(self.file_paths):
            data = np.load(file_path)
            num_crops = len(data['x_data'])
            crops_index.extend([(file_idx, crop_idx) for crop_idx in range(num_crops)])
        return crops_index

    def __len__(self):
        return len(self.crops_index)
    
    def __getitem__(self, idx):
        file_idx, crop_idx = self.crops_index[idx]
        file_path = self.file_paths[file_idx]

        data = np.load(file_path)
        crop = data['x_data'][crop_idx]
        label = data['y_data'][crop_idx]

        crop = torch.tensor(crop).float()
        label = torch.tensor(label).float().squeeze().unsqueeze(0)
        
        return crop, label

# Usage Example
precomputed_dir = "/home/marta/Documenti/eeg-ml-thesis/output/"
file_paths = [os.path.join(precomputed_dir, f) for f in os.listdir(precomputed_dir) if f.endswith('.npz')]

dataset = EegDataset(file_paths)

train_indices, val_indices = split_train_val(dataset, test_size=0.2)
train_loader = DataLoader(Subset(dataset, train_indices), batch_size=32, shuffle=True)
val_loader = DataLoader(Subset(dataset, val_indices), batch_size=32, shuffle=True)

# test_loader = DataLoader(test_dataset, batch_size = 32)